# Use PMML to predict iris species with `ibm-watson-machine-learning`

This notebook contains steps from storing sample PMML model to starting scoring new data. 

Some familiarity with python is helpful. This notebook uses Python 3.

You will use a **Iris** data set, which details measurements of iris perianth. Use the details of this data set to predict iris species.

## Learning goals

The learning goals of this notebook are:

-  Working with the WML instance
-  Online deployment of PMML model
-  Scoring of deployed model


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Model upload](#upload) 
3.	[Web service creation](#deploy)
4.	[Scoring](#score)
5.  [Clean up](#cleanup)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [29]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [3]:
space_id = 'PASTE YOUR SPACE ID HERE'

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [5]:
client.set.default_space(space_id)

'SUCCESS'

<a id="upload"></a>
## 2. Upload model

In this section you will learn how to upload the model to the Cloud.

**Action**: Download sample PMML model from git project using wget.

In [21]:
import os
from wget import download

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = download('https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/models/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store downloaded file in Watson Machine Learning repository.

In [22]:
sw_spec_uid = client.software_specifications.get_uid_by_name("spark-mllib_2.3")

meta_props = {
    client.repository.ModelMetaNames.NAME: "pmmlmodel",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid,
    client.repository.ModelMetaNames.TYPE: 'pmml_4.2.1'}

In [23]:
published_model = client.repository.store_model(model=filename, meta_props=meta_props)

**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

In [ ]:
client.repository.list_models()

<a id="deployment"></a>
## 3. Create online deployment

You can use commands bellow to create online deployment for stored model (web service).

In [25]:
model_uid = client.repository.get_model_uid(published_model)
deployment = client.deployments.create(
    artifact_uid=model_uid,
    meta_props={
        client.deployments.ConfigurationMetaNames.NAME: "Test deployment",
        client.deployments.ConfigurationMetaNames.ONLINE:{}}
)



#######################################################################################

Synchronous deployment creation for uid: 'e7c8f452-9fea-4d9c-90ff-4993ce08b024' started

#######################################################################################


initializing
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='264bb858-0f35-48a4-818f-3467fcd05d9c'
------------------------------------------------------------------------------------------------




<a id="scoring"></a>
## 4. Scoring

You can send new scoring records to web-service deployment using `score` method.

In [ ]:
deployment_id = client.deployments.get_id(deployment)

scoring_data = {
    client.deployments.ScoringMetaNames.INPUT_DATA: [
        {
            'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'],
            'values': [[5.1, 3.5, 1.4, 0.2]]
        }]
}

predictions = client.deployments.score(deployment_id, scoring_data)
print(predictions)

As we can see this is Iris Setosa flower.

<a id="cleanup"></a>
## 5. Clean up   

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Software Architect and Data Scientist at IBM.

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.